# Preprocessing Depth array

##### Deep learning Model outputs depth array of size 640 x 480 but which is different aspect ratio from the original images, therefore we resize the depth array along vertical axis using graph interpolation to get new depth array of size 640 x 426 

Imports

In [3]:
import numpy as np
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import os
from scipy import interpolate

In [4]:
def shrink_array(arr, new_size):
    '''
    Function to shrink 1D-array to a new size using graph interpolation
    '''
    old_size=arr.size
    x=np.arange(0,old_size)
    y=arr
    f = interpolate.interp1d(x, y)
    xnew=np.arange(0,old_size,old_size/new_size)
    ynew=f(xnew)
    #print("returning array with size {}".format(ynew.size))
    return ynew

def depth_resize(d_arr, new_size):
    '''
    Function to resize depth array 'd_arr' along vertical axis with new dimension as 'new_size'
    '''
    re_depth=np.zeros((new_size,640))
    for i in range(0,640):
        re_depth[:,i]=shrink_array(d_arr[:,i],new_size)
        
    return re_depth


def sparse_point_cloud(depth_map, size):
    '''
    Function to convert depth array into sparse point cloud
    '''
    fx, fz = 441, 443 #based on sensor size calculations
    cx, cz = 320, 213 #based on depth-map dimensions

    y = depth_map
    y =(-1)*y
    # convert depth image of shape to point map 
    # here we assume depth image is of shape (426, 640)
    px, pz = np.meshgrid(np.arange(640), np.arange(size))  # pixel_x, pixel_y
    px, pz = px.astype(float), pz.astype(float)
    x = ((px - cx) / fx) * y
    z = ((pz - cz) / fz) * y 
    pmap = np.concatenate([i[..., np.newaxis] for i in (x, y, z)], axis=-1)

    new_pmap=np.reshape(pmap, (size*640,-1))
    
    #truncating depth values greater than 5m
    spc=[]
    for points in new_pmap:
        if(abs(points[1])<5):
            spc.append(points)
    spc=np.array(spc)
    return spc

Saving sparse point clouds in a separate folder

In [5]:
depth_folder="Data/depth_txt"
sparse_folder="Data/spc_5m"
new_size=426

for d_file in os.listdir(depth_folder):
    depth=np.loadtxt(depth_folder+"/"+d_file)
    depth_map=depth_resize(depth, new_size)
    spc=sparse_point_cloud(depth_map, new_size)
    np.savetxt(sparse_folder+"/"+d_file, spc)